<a href="https://colab.research.google.com/github/Rotem2411/alephBERTgimmelDalet/blob/main/Latent_Dirichlet_Allocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import warnings

# Filter out DeprecationWarning
warnings.filterwarnings("ignore", category=DeprecationWarning, module="ipykernel")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
#if necessary install below
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 9.8 MB/s eta 0:00:00


In [4]:
import csv
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
import pyLDAvis
import pyLDAvis.gensim_models
from gensim.models.ldamodel import LdaModel
from gensim import corpora
from collections import defaultdict
from gensim import models
from pprint import pprint

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
file_path = 'sentiments.csv'
df = pd.read_csv(file_path)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Take a peek of the data

In [ ]:
num_samples = len(df)
print("Total number of samples: ", num_samples)
print(df.head(10))

Total number of samples:  75151
   id                                               text tag category class  \
0   1                    האריות של הצל חזק פה בתגובות...   ש  ECONOMY    b1   
1   2  זמרת תעשייה רק מפרסומות מפורסמת\n  אבל אין קהל...   ש  ECONOMY    b1   
2   4  את לא יורקת לבאר שממנה שתית. יפה יפה, אבל חסרת...   ש  ECONOMY    b1   
3   5  שמעון\n  משתתף בפרסומת למילקי במקום לעורר מודע...   ש  ECONOMY    b1   
4   6  הצחקתם אותי\n  מה כל כך אמייזינג בחברה עם אפס ...   ש  ECONOMY    b1   
5   7  איסו חברת הייטק מצליחה צריכה פרסומות שיבואו לע...   ש  ECONOMY    b1   
6   8           לא מכיר אותה ולא שמעתי אפילו שיר אחד שלה   ש  ECONOMY    b1   
7   9  וואללה לא יודע מה מתלהבים מנגה ארז. זה בכלל לא...   ש  ECONOMY    b1   
8  10  אהובה שגיא\n  נגה גדולה.זמרת מעולה.פרסומת מדלי...   ח  ECONOMY    b1   
9  11  הפרסומת של אמדוקס נראית מאוד קודרת ואפורה, כמו...   ש  ECONOMY    b1   

   total_tags  selected_tag  polarity  
0           2             2       1.0  
1           2     

# **Data Pre-processing**

We will perform the following steps:


*   **Tokenization**: Split the text into sentences and the sentences into words
*   Remove **punctuation**.
*   All **stopwords** are removed.
*   Words are **lemmatized** — words in third person are changed to first person and verbs in past and future tenses are changed into present.
*   Words are **hebrew** — words with english letters or numbers are removed.

In [8]:
def contains_english_letter(word):
    return any(c.isascii() for c in word)

In [9]:
def cleanwords(text):
    wn = nltk.WordNetLemmatizer()
    stopword = nltk.corpus.stopwords.words('hebrew')
    tokens = nltk.word_tokenize(text)
    no_stopwords = [word for word in tokens if word not in stopword]
    no_alpha = [word for word in no_stopwords if word.isalpha()]
    no_ascii = [word for word in no_alpha if not contains_english_letter(word)]
    lemm_text = [wn.lemmatize(word) for word in no_ascii]
    return lemm_text

In [10]:
text = df['text'].tolist()
clean_text = [cleanwords(t) for t in text]

In [11]:
text_comparison = pd.DataFrame({
    'text_before_preprocessed': text,
    'text_after_preprocessed': clean_text,
})
display(text_comparison.head(5))

,text_before_preprocessed,text_after_preprocessed
0,האריות של הצל חזק פה בתגובות...,"[האריות, הצל, חזק, בתגובות]"
1,זמרת תעשייה רק מפרסומות מפורסמת\n אבל אין קהל...,"[זמרת, תעשייה, מפרסומות, מפורסמת, קהל, אמיתי, ..."
2,"את לא יורקת לבאר שממנה שתית. יפה יפה, אבל חסרת...","[יורקת, לבאר, שממנה, שתית, יפה, יפה, חסרת, עמו..."
3,שמעון\n משתתף בפרסומת למילקי במקום לעורר מודע...,"[שמעון, משתתף, בפרסומת, למילקי, במקום, לעורר, ..."
4,הצחקתם אותי\n מה כל כך אמייזינג בחברה עם אפס ...,"[הצחקתם, אמייזינג, בחברה, אפס, נאמנות, לעובדים]"


Remove words that appear only once

In [12]:
frequency = defaultdict(int)
for i in range(len(clean_text)):
  for token in clean_text[i]:
    frequency[token] += 1

In [13]:
clean_text = [
    [token for token in text if frequency[token] > 1]
    for text in clean_text
]

In [14]:
dictionary = corpora.Dictionary(clean_text)
corpus = [dictionary.doc2bow(text) for text in clean_text]

# **TF-IDF**

In [15]:
tfidf = models.TfidfModel(corpus, normalize=True)
corpus_tfidf = tfidf[corpus]

# **LDA**

In [ ]:
ldamodel = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=4)
corpus_lda = ldamodel[corpus_tfidf]
ldamodel(corpus_tfidf)

Dominant topic

In [17]:
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus_tfidf, dictionary=ldamodel.id2word)
pyLDAvis.save_html(vis, 'LDA_Visualization.html')

In [74]:
topic_names = ["Topic" + str(i) for i in range(ldamodel.num_topics)]
doc_names = ["Doc" + str(i) for i in range(len(clean_text))]
df_topic_distribution = pd.DataFrame(data=corpus_tfidf, columns=topic_names, index=doc_names)
df_topic_distribution.head(10)

ValueError: 6 columns passed, passed data had 202 columns

In [29]:
# column names
topicnames = ["Topic" + str(i) for i in range(ldamodel.num_topics)]
# index names
docnames = ["Doc" + str(i) for i in range(len(clean_text))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(corpus_lda, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic
# Styling
def color_green(val):
  color = 'green' if val > .8 else 'black'
  return 'color: {col}'.format(col=color)
def make_bold(val):
  weight = 700 if val > .25 else 400
  return "font-weight: {weight}".format(weight=weight)
# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

ValueError: Must pass 2-d input. shape=(75151, 4, 2)

In [ ]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(ldamodel.num_topics)
# Assign Column and Index
df_topic_keywords.columns = tfidf_tokens
df_topic_keywords.index = topicnames
# View
df_topic_keywords.head()

,אאא,אאאא,אאאאוב,אאוהבםי,אאוט,אאוטלט,אאוטסורס,אאוטסורסינג,אאוטסיידר,אאוצ,...,תתרמי,תתרסק,תתרסקו,תתשדל,תתשפט,תתתנו,תתתת,ײסודה,בּטוח,בּמקצועם
Topic0,0.250096,0.250133,0.693101,0.425112,0.250220,0.250133,0.257378,0.250097,0.392831,0.250179,...,0.250118,0.870560,0.250170,0.250120,0.250104,0.252936,0.726342,0.429351,0.250103,0.250103
Topic1,0.250118,0.250164,0.250208,0.250111,0.573873,2.767795,0.250148,0.250103,0.250083,0.898556,...,0.250129,0.505283,0.842984,0.250147,0.250131,0.398933,0.250193,0.250110,0.250144,0.250144
Topic2,0.250116,0.761237,0.250208,0.250112,0.250315,0.250191,0.497816,0.543323,0.250095,0.250202,...,0.250144,1.167789,0.250185,0.681673,0.623669,0.250105,0.250216,0.250106,0.250116,0.250116
Topic3,1.244462,0.250183,0.250350,0.250128,0.603101,0.250168,0.250180,0.250124,0.250116,0.250256,...,0.692143,0.250441,0.250225,0.250157,0.250145,0.250151,0.250220,0.250117,0.655468,0.655468


Top 30 keywords each topic

In [ ]:
# Show top n keywords for each topic
def show_topics(vectorizer, lda_model, n_words):
    keywords = np.array(vectorizer.get_feature_names_out())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(tfidfvectorizer, lda_model, n_words=50)
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords).transpose()
df_topic_keywords.columns = ['Topic '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Word '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Topic 0,Topic 1,Topic 2,Topic 3
Word 0,צריך,כמה,צריך,כתבה
Word 1,שלא,כסף,ולא,טוב
Word 2,ביבי,עוד,כבר,לך
Word 3,כבר,ביבי,עוד,שלא
Word 4,ולא,ולא,ביבי,עוד
Word 5,לך,לך,שלא,צריך
Word 6,אחד,שלא,המדינה,כבר
Word 7,הכל,ישראל,ישראל,אחד
Word 8,הרבה,ממש,צודק,אנשים
Word 9,עוד,זו,הזה,ולא


In [ ]:
# Topic-Keyword Matrix
df_Topic_keywords = pd.DataFrame(lda_model.components_)
# Assign Column and Index
df_Topic_keywords.columns = tfidf_tokens
df_Topic_keywords.index = topicnames
# View
df_Topic_keywords.head()

,אאא,אאאא,אאאאוב,אאוהבםי,אאוט,אאוטלט,אאוטסורס,אאוטסורסינג,אאוטסיידר,אאוצ,...,תתרמי,תתרסק,תתרסקו,תתשדל,תתשפט,תתתנו,תתתת,ײסודה,בּטוח,בּמקצועם
Topic0,0.250096,0.250133,0.693101,0.425112,0.250220,0.250133,0.257378,0.250097,0.392831,0.250179,...,0.250118,0.870560,0.250170,0.250120,0.250104,0.252936,0.726342,0.429351,0.250103,0.250103
Topic1,0.250118,0.250164,0.250208,0.250111,0.573873,2.767795,0.250148,0.250103,0.250083,0.898556,...,0.250129,0.505283,0.842984,0.250147,0.250131,0.398933,0.250193,0.250110,0.250144,0.250144
Topic2,0.250116,0.761237,0.250208,0.250112,0.250315,0.250191,0.497816,0.543323,0.250095,0.250202,...,0.250144,1.167789,0.250185,0.681673,0.623669,0.250105,0.250216,0.250106,0.250116,0.250116
Topic3,1.244462,0.250183,0.250350,0.250128,0.603101,0.250168,0.250180,0.250124,0.250116,0.250256,...,0.692143,0.250441,0.250225,0.250157,0.250145,0.250151,0.250220,0.250117,0.655468,0.655468


 # **Best LDA model**

Using GridSearch to determine the best LDA model

In [ ]:
# Define Search Param
search_params = {'n_components': [3, 4, 5]}
# Init the Model
lda = LatentDirichletAllocation(max_iter=5,random_state=0)
# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)
# Do the Grid Search
model.fit(df_tfidfvect)

In [ ]:
# Best Model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best Model's Params: ", model.best_params_)
# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(df_tfidfvect))

Dominant topic

In [ ]:
# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]
# index names
docnames = ["Doc" + str(i) for i in range(len(text_after_preprocessed))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic
# Styling
def color_green(val):
  color = 'green' if val > .8 else 'black'
  return 'color: {col}'.format(col=color)
def make_bold(val):
  weight = 700 if val > .25 else 400
  return "font-weight: {weight}".format(weight=weight)
# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

SyntaxError: invalid character '“' (U+201C) (<ipython-input-15-038e417f1f42>, line 4)

In [ ]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)
# Assign Column and Index
df_topic_keywords.columns = tfidf_tokens
df_topic_keywords.index = topicnames
# View
df_topic_keywords.head()

In [ ]:
# Show top n keywords for each topic
def show_topics(vectorizer, lda_model, n_words):
    keywords = np.array(vectorizer.get_feature_names_out())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(tfidfvectorizer, best_lda_model, n_words=50)
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords).transpose()
df_topic_keywords.columns = ['Topic '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Word '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords